# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [71]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [72]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [73]:
show(boston, allcols=true)

506×14 DataFrame
│ Row │ Crim    │ Zn      │ Indus   │ Chas  │ NOx     │ Rm      │ Age     │
│     │ Float64 │ Float64 │ Float64 │ Int64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────┼─────────┼─────────┼───────┼─────────┼─────────┼─────────┤
│ 1   │ 0.00632 │ 18.0    │ 2.31    │ 0     │ 0.538   │ 6.575   │ 65.2    │
│ 2   │ 0.02731 │ 0.0     │ 7.07    │ 0     │ 0.469   │ 6.421   │ 78.9    │
│ 3   │ 0.02729 │ 0.0     │ 7.07    │ 0     │ 0.469   │ 7.185   │ 61.1    │
│ 4   │ 0.03237 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 6.998   │ 45.8    │
│ 5   │ 0.06905 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 7.147   │ 54.2    │
│ 6   │ 0.02985 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 6.43    │ 58.7    │
│ 7   │ 0.08829 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.012   │ 66.6    │
│ 8   │ 0.14455 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.172   │ 96.1    │
│ 9   │ 0.21124 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 5.631   │ 100.0   │
│ 10  │ 0.17004 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.004   │ 85.9 

In [74]:
for  name in names(boston)
    if name == :MedV || name == :Chas
        println(name)
        continue
    end
    boston[!,name] = convert.(Float64, boston[!,name])
    println(typeof(boston[!,name]))
    
    minVal = minimum(boston[name])
    maxVal = maximum(boston[name])
    boston[!,name] .-= minVal
    boston[!,name] ./= (maxVal-minVal)
end

Array{Float64,1}
Array{Float64,1}
Array{Float64,1}
Chas
Array{Float64,1}
Array{Float64,1}
Array{Float64,1}
Array{Float64,1}
Array{Float64,1}
Array{Float64,1}
Array{Float64,1}
Array{Float64,1}
Array{Float64,1}
MedV


┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[74]:9
└ @ Core ./In[74]:9
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[74]:10
└ @ Core ./In[74]:10


In [75]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))

506-element view(::Array{Int64,1}, [62, 369, 485, 278, 101, 111, 425, 383, 180, 25  …  127, 381, 332, 476, 360, 355, 294, 335, 313, 503]) with eltype Int64:
  62
 369
 485
 278
 101
 111
 425
 383
 180
  25
  12
 148
 296
   ⋮
  50
 259
 127
 381
 332
 476
 360
 355
 294
 335
 313
 503

In [76]:
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8)

([62, 369, 485, 278, 101, 111, 425, 383, 180, 25  …  56, 403, 166, 388, 38, 223, 231, 134, 59, 505], [145, 320, 452, 31, 139, 471, 274, 416, 339, 428  …  127, 381, 332, 476, 360, 355, 294, 335, 313, 503])

In [77]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64
1,0.0311726,0.0,0.70088,0,1.0,0.257137,0.977343,0.0196692
2,0.00527313,0.0,0.346041,0,0.32716,0.488983,0.575695,0.261192
3,0.0610861,0.0,0.646628,0,0.674897,0.592834,0.981462,0.11145
4,0.012639,0.0,0.281525,0,0.314815,0.41234,0.939238,0.282207
5,0.00273666,0.0,0.785557,0,0.49177,0.439931,0.981462,0.0490138
6,0.0488083,0.0,0.646628,0,0.401235,0.499329,0.835221,0.173122
7,0.00242284,0.2,0.23827,1,0.162551,0.791339,0.503605,0.294347
8,0.203196,0.0,0.646628,0,0.604938,0.550489,1.0,0.0641181
9,0.000300551,0.0,0.173387,0,0.26749,0.478636,0.354274,0.334876


In [78]:
model = GLM.lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
──────────────────────────────────────────────────────────────────────────────
               Estimate  Std. Error    t value  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   26.6453       2.79937   9.51831     <1e-18   21.1416    32.149
Crim          -6.52781      4.0853   -1.59788     0.1109  -14.5597     1.5041
Zn             4.62483      1.56277   2.95938     0.0033    1.55235    7.69731
Indus          0.707116     1.97285   0.358423    0.7202   -3.17161    4.58584
Chas           2.23628      1.06017   2.10936     0.0355    0.15193    4.32063
NOx           -9.30521      2.18109  -4.26632     <1e-4   -13.5933    -5.01708
Rm        

In [79]:
predict(model, test)

101-element Array{Union{Missing, Float64},1}:
  8.793640027884662
 21.588226478751942
 19.187052621670695
 11.686437181631296
 14.044907017328436
 19.968073354154324
 35.6808793309087
 10.240501149157586
 22.416058667579556
 14.905540826729352
 35.93727386952742
 13.070655068155874
 21.452969344561545
  ⋮
 17.43724375093793
 36.498747844917716
 15.352971864767973
 17.316069569699312
 20.334439660759063
 16.242567879400134
 18.635892900101634
 14.13081347631018
 26.33078607591082
 21.645632903453322
 23.257286866676655
 22.486111653573744

In [80]:
GLM.r2(model)

0.7365671732486647